# Imports

In [1]:
import os
import itertools

from dotenv import load_dotenv
load_dotenv()

import pandas as pd
pd.set_option('display.max_colwidth', None)

from utils import constants
from utils.process_data import Preprocessor
import utils.create as create

# Städte-Daten einlesen ODER Datensatz einlesen, filtern und auswählen

In [2]:
samples_path = './samples/all.csv'
# n = 1000 # Anzahl der ausgewählten Einträge

create.create_samples(samples_path, n=None)
samples = create.load_samples(samples_path)

print(samples.shape)
display(samples.head())

(3527, 6)


,index,Country,City,Population,Latitude,Longitude
0,93,ae,abu dhabi,603687,24.466667,54.366667
1,242,ae,dubai,1137376,25.258172,55.304717
2,490,ae,sharjah,543942,25.357310,55.403304
3,6644,af,baglan,108481,36.130684,68.708286
4,24457,af,gardez,103732,33.597439,69.225922


# Llama befragen

In [3]:
DEEPINFRA_API_KEY = os.getenv('DEEPINFRA_API_KEY')

file_name_template = '{model_key}-Instruct-{template_key}{add_country}'
path = './answers/'

queries = itertools.product(constants.models.keys(), constants.templates.keys(), [True, False])

# Für jede Kombination von LLM, Template und mit/ohne Land
for (model_key, template_key, add_country) in queries:
    file_name = file_name_template.format(model_key=model_key, template_key=template_key, add_country= '-country' if add_country else '')
    file_path = os.path.join(path, file_name + '.csv')

    create.query_save_parallel(constants.models[model_key], constants.templates[template_key], file_path, DEEPINFRA_API_KEY, samples, add_country=add_country, process_number=100)
    
    # Auswertung für das originale Template
    if template_key == 'original':
        path_output_llm = os.path.join(path, file_name + '-p_llm.csv')
        Preprocessor.llm_save_parallel(file_path, path_output_llm, DEEPINFRA_API_KEY, process_number=100)

        path_output_regex = os.path.join(path, file_name + '-p_regex.csv')
        Preprocessor.regex(file_path, path_output_regex)
    # Auswertung für das json-Template
    elif template_key == 'json':
        path_output_json = os.path.join(path, file_name + '-p.csv')
        Preprocessor.json(file_path, path_output_json)
    else:
        print(f"Die Datei {file_path} hat kein gültiges Template!")
    

Die Datei ./answers/llama-3.1-8B-Instruct-json-country.csv wurde vollständig erstellt!
Achtung! Die Datei ./answers/llama-3.1-8B-Instruct-json-country-p.csv existiert bereits.
Daher werden die zugehörigen Anfragen nicht ausgeführt. Es ist möglich, dass dieses Datei allerdings mit anderen Parametern und/oder Daten erzeugt wurde.
Die Datei ./answers/llama-3.1-8B-Instruct-json.csv wurde vollständig erstellt!
Achtung! Die Datei ./answers/llama-3.1-8B-Instruct-json-p.csv existiert bereits.
Daher werden die zugehörigen Anfragen nicht ausgeführt. Es ist möglich, dass dieses Datei allerdings mit anderen Parametern und/oder Daten erzeugt wurde.
Die Datei ./answers/llama-3.1-8B-Instruct-original-country.csv wurde vollständig erstellt!
Achtung! Die Datei ./answers/llama-3.1-8B-Instruct-original-country-p_llm.csv existiert bereits.
Daher werden die zugehörigen Anfragen nicht ausgeführt. Es ist möglich, dass dieses Datei allerdings mit anderen Parametern und/oder Daten erzeugt wurde.
Achtung! Die D